  # **Spellchecking system**





This assignment will involve the creation of a spellchecking system and an evaluation of its performance. 

Corpus:  `holbrook.txt` uploaded in Github.

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
## This Project consists of 5 following tasks:

Task 1: Creating a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. 

Task 2: Calculating the frequency (number of occurrences) of all words and their unigram probability from the corrected *training* sentences.

Task 3: Creating a function that calculates all words with *minimal* edit distance to the misspelled word. 

Task 4: Creating a function that takes a (misspelled) sentence and returns the corrected version of that sentence.

Task 5: Accuracy Check



## Task 1 

Creating a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. 

In [0]:
import nltk
#nltk.download("all")
lines = open("holbrook.txt").readlines()
data=[]
allLines=[]
for line in lines:
  dict={}
  allTextLines= line.split()                        #splitting by lines
  originalTextLines=line.split()
  correctTextLines=line.split()
  dict['indexes']=[]
  index=0
  for word in allTextLines:
    if '|' in word:
      dict['indexes'].append(index)
      originalTextLines[index]=word.split('|')[0]           #splitting wordsbased on pipe where leftmost word being wrong and rightmost being right 
      correctTextLines[index]=word.split('|')[1]
    index+=1
  dict['original']=originalTextLines
  dict['corrected']=correctTextLines
  data.append(dict) 
#print(data[2])
#print(corrected)
assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [0]:
test = data[:100]
train = data[100:]
#print(train)

[{'corrected': ['they', 'can', 'go', 'quite', 'fast'], 'original': ['they', 'can', 'go', 'quite', 'farst'], 'indexes': [4]}, {'corrected': ['this', 'was', 'a', 'Royal', 'Enfield', '_?_'], 'original': ['this', 'was', 'a', 'Royl', 'Enfield', 'Consulatoin'], 'indexes': [3, 5]}, {'corrected': ['there', 'were', 'some', 'Royal', 'Enfield', 'Racing', 'there', 'as', 'well'], 'original': ['there', 'were', 'some', 'Royl', 'Enfield', 'Racing', 'there', 'as', 'well'], 'indexes': [3]}, {'corrected': ['I', 'saw', 'a', 'Royal', 'Enfield', 'come', 'first', 'it', 'was', 'like', 'mine', '.'], 'original': ['I', 'saw', 'a', 'Royl', 'Enfield', 'come', 'first', 'it', 'was', 'like', 'mine', '.'], 'indexes': [3]}, {'corrected': ['there', 'were', 'the', 'new', 'Japanese', 'Honda'], 'original': ['there', 'were', 'the', 'new', 'Japannese', 'Hondor'], 'indexes': [4, 5]}, {'corrected': ['they', 'are', 'very', 'fast', 'and', 'getting', 'quite', 'popular', 'in', 'England'], 'original': ['they', 'are', 'very', 'farst

## **Task 2** 
Calculating the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.



In [0]:
from collections import Counter
#nltk.download("all")
#from nltk.tokenize import sent_tokenize,word_tokenize
corrected_train= list(i['corrected'] for i in train)

def unigram(word):
  counts=0
  for i in corrected_train:
    countList=Counter(i)
    counts+=countList[word]
  return (counts) 
   

def prob(word):
  count2=0.0
  for x in corrected_train:
    for y in x:
      count2=count2+1
  total= (unigram(word))/count2
  #print(total)
  return (total)
  
unigram("me")
prob("me")

# Test your code with the following
assert(unigram("me")==87)

## **Task 3**  

Creating a function that calculates all words with *minimal* edit distance to the misspelled word. 

[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. 

In [0]:
uniqueWords = [] 
for i in corrected_train:
  for x in i:
      if x not in uniqueWords:
          uniqueWords.append(x)

def get_candidates(token):
  distance=[]
  duplicate=[]
  min_list=[]
  index=0
  for k in uniqueWords:
    distance.append(nltk.edit_distance(k,token))    #calculating minimum distance of token with respect to every words in train data
    duplicate.append(k)
  for x in distance:
    if(x==min(distance)):
      min_list.append(duplicate[index])
    index+=1
  return(min_list)
  
   

get_candidates("minde")

# Test your code as follows

assert get_candidates("minde") == ['mine', 'mind']

## Task 4 :

Creating a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system scans the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 



In [0]:
def correct(sentence):
  min_dis=[]
  uni_prob=[]
  uni_words=[]
  lines = sentence
  index=0
  for word in lines:
    if word not in uniqueWords:
        min_dis=get_candidates(word)
        for word_distance in min_dis:
          uni_prob.append(prob(word_distance))
          #uni_prob.append(b)    
          uni_words.append(word_distance)
          index1=0
        for i in uni_prob:
          if(i==max(uni_prob)):
            lines[index]=uni_words[index1]
          index1+=1
    index+=1
  return lines

correct(["this","whitr","cat"])

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5** :

Using the test corpus to evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence 

In [0]:
def accuracy(test):
  corrected_list=[]
  original_test= list(i['original'] for i in test)
  corrected_test= list(i['corrected'] for i in test)
  words = [word.lower() for line in corrected_test for word in line]
  for line in original_test:
    corrected_list.append(correct(line))
  counter=0
  count=0
  for line in corrected_list:
    for x in line:
      #print(x in corrected_test)
      if x in words:
        count+=1
  for lines in corrected_test:
    for j in lines:
      counter+=1
  accuracy=float(count)/counter*100
  
  return(accuracy)
    
  
    
#accuracy(test)

print(accuracy(test))


81.0451727192
